In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
import nltk
from nltk.corpus import stopwords
from nltk.stem import wordnet
from nltk import pos_tag
from nltk import word_tokenize
import re

In [14]:
df=pd.read_excel('c:/users/administrator/downloads/chatbot_data.xlsx')

In [15]:
df.head()

,Context,Text Response
0,Tell me about your personality,Just think of me as the ace up your sleeve.
1,I want to know you better,I can help you work smarter instead of harder
2,Define yourself,NaN
3,Describe yourself,NaN
4,tell me about yourself,NaN


In [16]:
df.ffill(axis = 0,inplace=True) 

In [17]:
df.head()

,Context,Text Response
0,Tell me about your personality,Just think of me as the ace up your sleeve.
1,I want to know you better,I can help you work smarter instead of harder
2,Define yourself,I can help you work smarter instead of harder
3,Describe yourself,I can help you work smarter instead of harder
4,tell me about yourself,I can help you work smarter instead of harder


In [18]:
df1=df.head(10)

In [19]:
def step1(x):
    for i in x:
        a=str(i).lower()
        p=re.sub(r'[^a-z0-9]',' ',a)
        print(p)

In [20]:
step1(df1['Context'])


tell me about your personality
i want to know you better
define yourself
describe yourself
tell me about yourself
all about you
tell me some stuff about you
talk some stuff about you
talk about yourself
about yourself


In [21]:
s='tell me about your personality'
words=word_tokenize(s)
print(words)

['tell', 'me', 'about', 'your', 'personality']


In [22]:

lemma = wordnet.WordNetLemmatizer() 
lemma.lemmatize('absorbed', pos = 'v')

'absorb'

In [23]:
pos_tag(nltk.word_tokenize(s),tagset = None)

[('tell', 'VB'),
 ('me', 'PRP'),
 ('about', 'IN'),
 ('your', 'PRP$'),
 ('personality', 'NN')]

In [24]:
def text_normalization(text):
    text=str(text).lower() 
    spl_char_text=re.sub(r'[^ a-z]','',text) 
    tokens=nltk.word_tokenize(spl_char_text)
    lema=wordnet.WordNetLemmatizer()
    tags_list=pos_tag(tokens,tagset=None) 
    lema_words=[]   
    for token,pos_token in tags_list:
        if pos_token.startswith('V'):  
            pos_val='v'
        elif pos_token.startswith('J'): 
            pos_val='a'
        elif pos_token.startswith('R'): 
            pos_val='r'
        else:
            pos_val='n' 
        lema_token=lema.lemmatize(token,pos_val) 
        lema_words.append(lema_token) 
    
    return " ".join(lema_words)

In [25]:
text_normalization('telling you some stuff about me')


'tell you some stuff about me'

In [26]:
df['lemmatized_text']=df['Context'].apply(text_normalization) 
df.tail(15)

,Context,Text Response,lemmatized_text
1577,I need to talk to you,Good conversation really makes my day.,i need to talk to you
1578,I want to speak with you,I'm always here to lend an ear.,i want to speak with you
1579,let's have a discussion,Talking is what I do best.,let have a discussion
1580,I just want to talk,Talking is what I do best.,i just want to talk
1581,let's discuss something,Talking is what I do best.,let discuss something
1582,can I speak,Talking is what I do best.,can i speak
1583,can we talk,Talking is what I do best.,can we talk
1584,let's talk,Talking is what I do best.,let talk
1585,I want to talk to you,Talking is what I do best.,i want to talk to you
1586,can we chat,Talking is what I do best.,can we chat


In [27]:

stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [28]:

cv = CountVectorizer() 
X = cv.fit_transform(df['lemmatized_text']).toarray()

In [33]:
features = cv.get_feature_names()
df_bow = pd.DataFrame(X, columns = features)
df_bow.head()

,abort,about,absolutely,abysmal,actually,adore,advice,advise,affirmative,afraid,...,yeh,yep,yes,yet,you,your,youre,yours,yourself,yup
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [34]:
Question ='Will you help me and tell me about yourself more'

In [35]:
Q=[]
a=Question.split()
for i in a:
    if i in stop:
        continue
    else:
        Q.append(i)
    b=" ".join(Q)

In [37]:
Question_lemma = text_normalization(b) 
Question_bow = cv.transform([Question_lemma]).toarray()

In [38]:
cosine_value =cosine_similarity(df_bow, Question_bow)
(cosine_value)

array([[0.25819889],
       [0.        ],
       [0.        ],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]])

In [39]:
df['similarity_bow']=cosine_value

In [43]:
df_simi = pd.DataFrame(df, columns=['Text Response','similarity_bow'])
df_simi.head()

,Text Response,similarity_bow
0,Just think of me as the ace up your sleeve.,0.258199
1,I can help you work smarter instead of harder,0.000000
2,I can help you work smarter instead of harder,0.000000
3,I can help you work smarter instead of harder,0.000000
4,I can help you work smarter instead of harder,0.288675


In [44]:
df_simi_sort = df_simi.sort_values(by='similarity_bow', ascending=False)
df_simi_sort.head()

,Text Response,similarity_bow
211,I'm glad to help. What can I do for you?,0.577350
194,I'm glad to help. What can I do for you?,0.577350
184,I'm glad to help. What can I do for you?,0.408248
186,I'm glad to help. What can I do for you?,0.408248
200,I'm glad to help. What can I do for you?,0.408248


In [48]:
threshold = 0.2
df_threshold = df_simi_sort[df_simi_sort['similarity_bow'] > threshold] 
df_threshold.head()

,Text Response,similarity_bow
211,I'm glad to help. What can I do for you?,0.577350
194,I'm glad to help. What can I do for you?,0.577350
184,I'm glad to help. What can I do for you?,0.408248
186,I'm glad to help. What can I do for you?,0.408248
200,I'm glad to help. What can I do for you?,0.408248


In [49]:

index_value = cosine_value.argmax()
index_value

194

In [50]:
(Question)


'Will you help me and tell me about yourself more'

In [51]:
df['Text Response'].loc[index_value]

"I'm glad to help. What can I do for you?"

In [81]:
def stopword_(text):   
    tag_list=pos_tag(nltk.word_tokenize(text),tagset=None)
#     print(tag_list)
    stop=stopwords.words('english')
    lema=wordnet.WordNetLemmatizer()
    lema_word=[]
    for token,pos_token in tag_list:
        if token in stop:
            continue
        if pos_token.startswith('V'):
            pos_val='v'
        elif pos_token.startswith('J'):
            pos_val='a'
        elif pos_token.startswith('R'):
            pos_val='r'
        else:
            pos_val='n'
        lema_token=lema.lemmatize(token,pos_val)
#         print(lema_token)
        lema_word.append(lema_token)
    return " ".join(lema_word)

In [84]:
def chat_bow(text):
    s=stopword_(text)
    print(s)
    lemma=text_normalization(s) 
    print(lemma)
    bow=cv.transform([lemma]).toarray() # applying bow
    cosine_value = cosine_similarity(df_bow,bow)
    index_value=cosine_value.argmax() # getting index value 
    return df['Text Response'].loc[index_value]

In [92]:
chat_bow("hi")


hi
hi


'Hey!'

In [93]:
chat_bow("i miss you")

miss
miss


"I've been right here all along!"

In [94]:
chat_bow("you are amazing")

amaze
amaze


'Terrific!'